In [ ]:
import gradio as gr
import pandas as pd
import difflib  # for fuzzy matching
import re
from google.generativeai import GenerativeModel

# -----------------------------
# Data Setup
# -----------------------------
# Load wine data
df = pd.read_csv("../Resources/processed_wine_data.csv")

# Extract unique wine colors for the dropdown
color_options = df["Color"].dropna().unique().tolist()

# -----------------------------

# -----------------------------
# This instance is used for both wine recommendations and food pairing generation.
model = GenerativeModel("gemini-pro")

# -----------------------------
# Helper Functions
# -----------------------------
def get_dynamic_options(color):
    filtered_df = df[df["Color"] == color]
    # For 'aromas' and 'Flavor', assume comma-separated strings.
    aroma_options = (
        filtered_df["aromas"]
        .dropna()
        .apply(lambda x: [s.strip() for s in x.split(",")])
        .explode()
        .unique()
        .tolist()
    )
    flavor_options = (
        filtered_df["Flavor"]
        .dropna()
        .apply(lambda x: [s.strip() for s in x.split(",")])
        .explode()
        .unique()
        .tolist()
    )
    body_options = filtered_df["Body_Type"].dropna().unique().tolist()
    alcohol_options = filtered_df["Alcohol_Content"].dropna().unique().tolist()
    price_options = filtered_df["Price_Interval"].dropna().unique().tolist()
    
    return aroma_options, flavor_options, body_options, alcohol_options, price_options

def similarity(a, b):
    """Return a similarity ratio between two strings (case-insensitive)."""
    return difflib.SequenceMatcher(None, str(a).lower(), str(b).lower()).ratio()

# -----------------------------
# Wine Recommendation Function
# -----------------------------
def recommend_wine(color, aroma, flavor, body, alcohol, price):
    # Try to find an exact match:
    exact_df = df[
        (df["Color"] == color) &
        (df["aromas"].str.contains(aroma, na=False)) &
        (df["Flavor"].str.contains(flavor, na=False)) &
        (df["Body_Type"] == body) &
        (df["Alcohol_Content"] == alcohol) &
        (df["Price_Interval"] == price)
    ]
    
    if not exact_df.empty:
        selected_wine = exact_df.sample(1).iloc[0]
        prompt = f"""
Based on the following wine attributes:
Color: {color}
Aroma: {aroma}
Flavor: {flavor}
Body: {body}
Alcohol Content: {alcohol}
Price Range: {price}

Recommend a wine from this dataset and provide a brief description:
"""
        ai_response = model.generate_content(prompt)
        recommendation = ai_response.text if ai_response else "No AI response received."
        return (f"Wine Name: {selected_wine['Name']}\n\n"
                f"Description: {recommendation}\n\n"
                f"More info: {selected_wine['URL']}")
    else:
        # Fuzzy matching (restrict to wines with the same color)
        candidate_df = df[df["Color"] == color].copy()
        if candidate_df.empty:
            return "No wines available for the selected color."
        
        scores = []
        for idx, row in candidate_df.iterrows():
            aroma_score = similarity(row["aromas"], aroma)
            flavor_score = similarity(row["Flavor"], flavor)
            body_score = similarity(row["Body_Type"], body)
            alcohol_score = similarity(row["Alcohol_Content"], alcohol)
            price_score = similarity(row["Price_Interval"], price)
            overall_score = (aroma_score + flavor_score + body_score + alcohol_score + price_score) / 5
            scores.append(overall_score)
        
        candidate_df['score'] = scores
        best_match = candidate_df.sort_values(by='score', ascending=False).iloc[0]
        
        prompt = f"""
Exact match not found. Using fuzzy matching, here is the closest wine based on your selections:
Color: {color}
Aroma: {aroma}
Flavor: {flavor}
Body: {body}
Alcohol Content: {alcohol}
Price Range: {price}

Provide a brief description for this wine:
"""
        ai_response = model.generate_content(prompt)
        recommendation = ai_response.text if ai_response else "No AI response received."
        return (f"Closest Wine Match: {best_match['Name']} (Score: {best_match['score']:.2f})\n\n"
                f"Description: {recommendation}\n\n"
                f"More info: {best_match['URL']}")

# -----------------------------
# Food Pairing Generation Function
# -----------------------------
def generate_food_pairing(color, aroma, flavor):
    prompt = (f"Provide me with a suggested meal that pairs well with a {color} wine "
              f"that has {flavor} flavor notes and {aroma} aromas. "
              "Include the recipe to cook the meal and the shopping list.")
    
    ai_response = model.generate_content(prompt)
    if not ai_response:
        return "No response from the food pairing model."
    
    result = ai_response.text.replace("*", "")
    
    # Parse the text into sections using regex.
    intro_match = re.search(r"^(.*?)Recipe:", result, re.DOTALL)
    recipe_match = re.search(r"Recipe:\s*(.*?)Shopping List:", result, re.DOTALL)
    shopping_list_match = re.search(r"Shopping List:\s*\n([\s\S]+)", result)
    
    intro_text = intro_match.group(1).strip() if intro_match else ""
    recipe_text = recipe_match.group(1).strip() if recipe_match else ""
    
    shopping_list_text = shopping_list_match.group(1).strip() if shopping_list_match else ""
    shopping_list_lines = shopping_list_text.split("\n")
    
    shopping_list = []
    extra_content = []
    found_extra = False
    for line in shopping_list_lines:
        if found_extra or not re.match(r"^\s*[\w\d(-]", line):
            found_extra = True
            extra_content.append(line)
        else:
            shopping_list.append(line.strip())
    extra_text = "\n".join(extra_content).strip()
    
    output = "INTRODUCTION:\n" + intro_text + "\n\n"
    output += "RECIPE & INGREDIENTS:\n" + recipe_text + "\n\n"
    output += "SHOPPING LIST:\n"
    for item in shopping_list:
        output += f"* {item}\n"
    if extra_text:
        output += "\n" + extra_text
    return output

# -----------------------------
# Update Dropdowns Function
# -----------------------------
def update_dropdowns(color):
    """
    Given the selected wine color, update the other dropdowns with their dynamic choices.
    This version inserts a blank option at the top so that no field auto-populates.
    """
    if not color:
        # If no color is selected, return empty dropdowns.
        return (
            gr.update(choices=[], value=""),
            gr.update(choices=[], value=""),
            gr.update(choices=[], value=""),
            gr.update(choices=[], value=""),
            gr.update(choices=[], value="")
        )
    
    aroma_options, flavor_options, body_options, alcohol_options, price_options = get_dynamic_options(color)
    # Insert a blank option at the top of each list so the field remains unselected.
    return (
        gr.update(choices=[""] + aroma_options, value=""),
        gr.update(choices=[""] + flavor_options, value=""),
        gr.update(choices=[""] + body_options, value=""),
        gr.update(choices=[""] + alcohol_options, value=""),
        gr.update(choices=[""] + price_options, value="")
    )

# -----------------------------
# Gradio Interface Setup
# -----------------------------
# The custom CSS below sets the width for any element with class "smaller-dropdown".
custom_css = """
.smaller-dropdown { 
    width: 150px !important; 
    margin-right: 10px;
}
"""

with gr.Blocks(css=custom_css) as iface:
    gr.Markdown("# Wine Recommendation & Food Pairing App")
    
    # Arrange dropdowns horizontally using gr.Row
    with gr.Row():
        color_input = gr.Dropdown(
            choices=color_options,
            label="Wine Color",
            value=None,  # No default selection
            elem_classes="smaller-dropdown"
        )
        aroma_input = gr.Dropdown(choices=[], label="Aroma", elem_classes="smaller-dropdown")
        flavor_input = gr.Dropdown(choices=[], label="Flavor", elem_classes="smaller-dropdown")
        body_input = gr.Dropdown(choices=[], label="Body", elem_classes="smaller-dropdown")
        alcohol_input = gr.Dropdown(choices=[], label="Alcohol", elem_classes="smaller-dropdown")
        price_input = gr.Dropdown(choices=[], label="Price", elem_classes="smaller-dropdown")
    
    # Output boxes
    recommend_output = gr.Textbox(label="Wine Recommendation", lines=10)
    food_pairing_output = gr.Textbox(label="Food Pairing Recommendation", lines=15)
    
    # Update dropdowns when a wine color is selected.
    color_input.change(
        fn=update_dropdowns, 
        inputs=color_input,
        outputs=[aroma_input, flavor_input, body_input, alcohol_input, price_input]
    )
    
    # Buttons for actions.
    recommend_button = gr.Button("Get Wine Recommendation")
    food_pairing_button = gr.Button("Get Food Pairing Recommendation")
    
    # Wire up the buttons.
    recommend_button.click(
        fn=recommend_wine, 
        inputs=[color_input, aroma_input, flavor_input, body_input, alcohol_input, price_input],
        outputs=recommend_output
    )
    
    food_pairing_button.click(
        fn=generate_food_pairing,
        inputs=[color_input, aroma_input, flavor_input],
        outputs=food_pairing_output
    )


iface.launch()

* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.
